In [1]:
# 環境構築

!apt-get update
!apt-get install -y libsndfile1-dev
!pip install scipy
!pip install scikit-learn
!pip install librosa

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease     
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Fetched 222 kB in 2s (90.4 kB/s)                                  
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1-dev is already the newest version (1.0.28-7ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
# clips 作成

SAMPLING_RATE = 44100 # 変更不可

clip_size = 81920 # 楽曲を再構築するパーツ1つあたりの大きさ
window_size = 20480 # CQT_CHROMA を取得するのに使用するサンプル数
hop_size = 10240 # window をずらすときの大きさ
window_num_per_clip = int(clip_size / window_size)

from scipy.io.wavfile import read, write
import glob
import os
import numpy as np
import librosa
import soundfile as sf
import sys

if os.path.isfile("../data/out/clips.wav"):
    print("../data/out/clips.wav exists.")
    files = ["../data/out/clips.wav"]
else:
    # clips.npy をもとにデータを作成する 
    if os.path.isfile("../data/arrays/clips.npy"):
        print("loading ../data/arrays/clips.npy ...")
        clips = np.load("../data/arrays/clips.npy")
        print("creating ../data/out/clips.wav ...")
        write("../data/out/clips.wav", SAMPLING_RATE, clips.reshape((clips.shape[0] * clips.shape[1], )))
        files = ["../data/out/clips.wav"]
    # clips,npy がないとき /data/wav44100 内の WAV ファイルを参照する
    else:
        #files = glob.glob("../data/wav44100/*.wav")
        files = ["../data/wav44100/3DEmbodimentFromLines.wav"] # デバッグ用 
        
if (len(files) == 0):
    print("WAV ファイルが見つかりませんでした。")
    sys.exit(1)
        
clips_filename = "../data/arrays/c" + str(clip_size) + "_f32_clips"

if os.path.isfile(clips_filename + ".npy"):
    print("loading " + clips_filename + ".npy ...")
    clips = np.load(clips_filename + ".npy")
else:
    print("creating " + clips_filename + ".npy ...")
    raw_data_list = [librosa.load(file, sr=SAMPLING_RATE)[0] for file in files] # 左の音だけ使う

    clips = np.zeros((0, clip_size), dtype=np.float32)
    for raw_data in raw_data_list:
        tmp = [raw_data[i:i + clip_size] for i in range(0, len(raw_data) - clip_size, clip_size)]
        clips = np.vstack((clips, np.array(tmp)))
    np.save(clips_filename, clips)

print("The clip array has " + str(clips.shape[0]) + " clips.")

creating ../data/arrays/c81920_f32_clips.npy ...
The clip array has 127 clips.


In [3]:
# cqt 作成

n_bins = 84

cqt_filename = clips_filename + "_w" + str(window_size) + "_h" + str(hop_size) + "_cqts"

if os.path.isfile(cqt_filename + ".npy"):
    print("loading " + cqt_filename + ".npy ...")
    cqts = np.load(cqt_filename + ".npy")
else:
    print("creating " + cqt_filename + ".npy ...")
    num = 0
    for clip in clips:
        print("cqt progress: clip " + str(num + 1) + " / " + str(len(clips)))
        
        tmp_cqt = librosa.cqt(clip, sr=SAMPLING_RATE, hop_length=hop_size, n_bins=n_bins)
        tmp_cqt = tmp_cqt.reshape((1, tmp_cqt.shape[0], tmp_cqt.shape[1]))
        
        if (num == 0):
            cqts = tmp_cqt
        else:
            cqts = np.vstack((cqts, tmp_cqt))
            
        num += 1

    np.save(cqt_filename, cqts)
    
print("The cqt array has " + str(cqts.shape[0]) + " cqts.")
print("cqts.shape: " + str(cqts.shape))
print("Type(cqts[0][0][0]): " + str(type(cqts[0][0][0])))
print("np.max(cqts[0][0]): " + str(np.max(cqts[0][0])))

creating ../data/arrays/c81920_f32_clips_w20480_h10240_cqts.npy ...
cqt progress: clip 1 / 127
cqt progress: clip 2 / 127
cqt progress: clip 3 / 127
cqt progress: clip 4 / 127
cqt progress: clip 5 / 127
cqt progress: clip 6 / 127
cqt progress: clip 7 / 127
cqt progress: clip 8 / 127
cqt progress: clip 9 / 127
cqt progress: clip 10 / 127
cqt progress: clip 11 / 127
cqt progress: clip 12 / 127
cqt progress: clip 13 / 127
cqt progress: clip 14 / 127
cqt progress: clip 15 / 127
cqt progress: clip 16 / 127
cqt progress: clip 17 / 127
cqt progress: clip 18 / 127
cqt progress: clip 19 / 127
cqt progress: clip 20 / 127
cqt progress: clip 21 / 127
cqt progress: clip 22 / 127
cqt progress: clip 23 / 127
cqt progress: clip 24 / 127
cqt progress: clip 25 / 127
cqt progress: clip 26 / 127
cqt progress: clip 27 / 127
cqt progress: clip 28 / 127
cqt progress: clip 29 / 127
cqt progress: clip 30 / 127
cqt progress: clip 31 / 127
cqt progress: clip 32 / 127
cqt progress: clip 33 / 127
cqt progress: cli

In [4]:
# cqt_choroma 作成

def Preprocessing(array):
    array = np.abs(array)
    array = np.log(array + 1)
    array = array.T
    return array

cqt_chroma_filename = clips_filename + "_w" + str(window_size) + "_h" + str(hop_size) + "_cqt_chromas"

if os.path.isfile(cqt_chroma_filename + ".npy"):
    print("loading " + cqt_chroma_filename + ".npy ...")
    cqt_chromas= np.load(cqt_chroma_filename + ".npy")
else:
    print("creating " + cqt_chroma_filename + ".npy ...")
    num = 0
    for cqt in cqts:
        print("cqt_chroma progress: clip " + str(num + 1) + " / " + str(len(cqts)))
        
        tmp_cqt_chroma = librosa.feature.chroma_cqt(C=cqt, sr=SAMPLING_RATE)
        tmp_cqt_chroma = Preprocessing(tmp_cqt_chroma)
        tmp_cqt_chroma = tmp_cqt_chroma.reshape((1, tmp_cqt_chroma.shape[0], tmp_cqt_chroma.shape[1]))
        
        if (num == 0):
            cqt_chromas = tmp_cqt_chroma
        else:
            cqt_chromas = np.vstack((cqt_chromas, tmp_cqt_chroma))
            
        num += 1

    np.save(cqt_chroma_filename, cqt_chromas)
    
print("The cqt_chroma array has " + str(cqt_chromas.shape[0]) + " cqt_chromas.")
print("cqt_chromas.shape: " + str(cqt_chromas.shape)) # clip 番号、window 番号、 chroma 番号 になる
print("Type(cqt_chromas[0][0][0]): " + str(type(cqt_chromas[0][0][0])))
print("np.max(cqt_chromas[0][0]): " + str(np.max(cqt_chromas[0][0])))

creating ../data/arrays/c81920_f32_clips_w20480_h10240_cqt_chromas.npy ...
cqt_chroma progress: clip 1 / 127
cqt_chroma progress: clip 2 / 127
cqt_chroma progress: clip 3 / 127
cqt_chroma progress: clip 4 / 127
cqt_chroma progress: clip 5 / 127
cqt_chroma progress: clip 6 / 127
cqt_chroma progress: clip 7 / 127
cqt_chroma progress: clip 8 / 127
cqt_chroma progress: clip 9 / 127
cqt_chroma progress: clip 10 / 127
cqt_chroma progress: clip 11 / 127
cqt_chroma progress: clip 12 / 127
cqt_chroma progress: clip 13 / 127
cqt_chroma progress: clip 14 / 127
cqt_chroma progress: clip 15 / 127
cqt_chroma progress: clip 16 / 127
cqt_chroma progress: clip 17 / 127
cqt_chroma progress: clip 18 / 127
cqt_chroma progress: clip 19 / 127
cqt_chroma progress: clip 20 / 127
cqt_chroma progress: clip 21 / 127
cqt_chroma progress: clip 22 / 127
cqt_chroma progress: clip 23 / 127
cqt_chroma progress: clip 24 / 127
cqt_chroma progress: clip 25 / 127
cqt_chroma progress: clip 26 / 127
cqt_chroma progress: cli